In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Dropout, LeakyReLU, DepthwiseConv2D, Flatten
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

from tqdm import tqdm, tqdm_notebook

import cv2 as cv
import seaborn as sns
import matplotlib.pyplot as plt

import gc


import os
print(os.listdir("../input"))

In [ ]:
train_df = pd.read_csv('../input/train.csv')
sub_df = pd.read_csv('../input/sample_submission.csv')

In [ ]:
train_df.head()
train_df.has_cactus.value_counts()

In [ ]:
training_path = '../input/train/train/'
test_path = '../input/test/test/'

images_train = []
labels_train = []

images = train_df['id'].values
for image_id in tqdm_notebook(images):
    
    image = np.array(cv.imread(training_path + image_id))
    label = train_df[train_df['id'] == image_id]['has_cactus'].values[0]
    
    images_train.append(image)
    labels_train.append(label)
    
    
    # custom image augmentation by fliping the image upside down
    images_train.append(np.flip(image))
    labels_train.append(label)
    
    # custom image augmentation by fliping the image upside down
    images_train.append(np.flipud(image))
    labels_train.append(label)
    
    # custom image augmentation by fliping the image diagonally
    images_train.append(np.fliplr(image))
    labels_train.append(label)
    
    
images_train = np.asarray(images_train)
images_train = images_train.astype('float32')
images_train /= 255.

labels_train = np.asarray(labels_train)

In [ ]:
def create_model():
    
    from numpy.random import seed
    seed(42)
    from tensorflow import set_random_seed
    set_random_seed(42)
    
    model = Sequential()
        
    model.add(Conv2D(3, kernel_size = 3, activation = 'relu', input_shape = (32, 32, 3)))
    
    model.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
    model.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    model.add(Conv2D(filters = 128, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters = 128, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 256, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters = 256, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 512, kernel_size = 3, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(470, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(128, activation = 'tanh'))
    
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(Adam(lr = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
test_images_names = []

for filename in os.listdir(test_path):
    test_images_names.append(filename)
    
test_images_names.sort()

images_test = []

for image_id in tqdm_notebook(test_images_names):
    images_test.append(np.array(cv.imread(test_path + image_id)))
    
images_test = np.asarray(images_test)
images_test = images_test.astype('float32')
images_test /= 255

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(images_train, labels_train, test_size = 0.2, stratify = labels_train)

In [ ]:
model = create_model()

In [ ]:
def plot_training_curves(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Losses')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, acc, 'r', label='Training acc')
    plt.plot(epochs, val_acc, 'g', label='Validation acc')
    plt.title('Accuracies')
    plt.legend()
    plt.figure()
    
    plt.show()

In [ ]:
batch_size = 64 
epochs = 50

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(0.01), 
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(monitor='val_acc', mode='max', filepath='model.hdf5', verbose=2, save_best_only=True)
earlyStopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='max')

history = model.fit(x = x_train, y = y_train, validation_data=(x_valid, y_valid), epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, earlyStopping])

In [ ]:
model.load_weights("model.hdf5")

In [ ]:
predictions = model.predict(images_test, verbose = 1)

In [ ]:
plot_training_curves(history)

In [ ]:
y_pred_probability = model.predict_proba(images_train)

y_pred = model.predict_classes(images_train)
conf_matrix = confusion_matrix(labels_train, y_pred)
fig, ax = plt.subplots(figsize = (10, 10))

sns.heatmap(conf_matrix, annot = True, fmt = 'd', xticklabels = ['0', '1'], yticklabels = ['0', '1'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

print(classification_report(labels_train, y_pred, target_names = ['0','1']))
print("\n\n AUC: {:<0.4f}".format(roc_auc_score(labels_train, y_pred_probability)))

In [ ]:
sub_df = pd.read_csv('../input/sample_submission.csv')
X_test = []
images_test = sub_df['id'].values

for img_id in tqdm_notebook(images_test):
    X_test.append(cv.imread(test_path + img_id))
    
X_test = np.asarray(X_test)
X_test = X_test.astype('float32')
X_test /= 255

y_test_pred = model.predict_proba(X_test)

sub_df['has_cactus'] = y_test_pred
sub_df.to_csv('aerial-cactus-submission.csv', index = False)